In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.special import gamma as gamma_function


In [2]:
total_nodes = 200
anchor_percent = 0.50
N = int((1 - anchor_percent) * total_nodes) # unknown nodes
M = total_nodes - N # anchor nodes

In [3]:
def alpha(n_iter):
    return alpha_max - ((n_iter/ N_itertotal) * (alpha_max - alpha_min))

# Levy Flight
def levy():
    # lamda = np.random.uniform(1.1,3)
    lamda = 1.5
    temp = np.power(((gamma_function(1 + lamda) * np.sin(np.pi * (lamda /2))) / (gamma_function((1 + lamda)/2) * lamda * np.power(2, ((lamda - 1)/2)) )), 1/lamda)
    u = np.random.normal(0, temp)
    v = np.random.normal(0,1)
    r = u / (np.power(abs(v), (1/lamda)))

    return r

In [4]:
    #objective function need to minimize
    def objective(x, y):
        l = []
        for j in range(len(X_anchor)): # for every anchor nodes
            dij = np.power((np.power((x - X_anchor[j][0]), 2) + np.power((y - X_anchor[j][1]), 2)), 0.5)
            dij_dash = dij + np.random.normal(0, (gamma*dij))

            if dij < t_range:
                l.append(X_anchor[j])
        if len(l) >= 3:
            m = len(l)
            rerror = []
            for ancn in l:
                dij = np.power((np.power((x - ancn[0]), 2) + np.power((y - ancn[1]), 2)), 0.5)
                dij_dash = dij + (gamma * gamma * dij *dij)
                rerror.append(np.power((dij - dij_dash), 2))

            return np.sum(rerror) / m

In [5]:
n_iters = []
n_unknown = []
lsr_values = []

In [ ]:
for case in range(30):
    print(case)
    #anchor nodes and unknown nodes random placement
    X_unknown = []
    X_anchor = []

    for i in range(N):
        x_unknown = np.random.randint(100)
        y_unknown = np.random.randint(100)

        X_unknown.append([x_unknown, y_unknown])

    for i in range(M):
        x_anchor = np.random.randint(100)
        y_anchor = np.random.randint(100)

        X_anchor.append([x_anchor, y_anchor])




    X_unknown = np.array(X_unknown)
    X_anchor = np.array(X_anchor)
    X_anchor_og = X_anchor.copy()
    X_unknown_bak = X_unknown.copy()
    
    t_range = 25 # transmission range
    gamma = 0.1 # noise factor
    

        
    v_x = []
    for i in range(100):
        for j in range(100):
            if objective(i, j):
                v_x.append(objective(i, j))
                
    threshold = np.min(v_x) + 10
    
    alpha_min = 0.9
    alpha_max = 1.0
    pa_min = 0.05
    pa_max = 0.25
    X_min = 0
    X_max = 100
    
    N_itertotal = 100
    
    
    
    ales = []
    X_calc = [] # final coordinates

    N_iter = 1
    while (N_iter < N_itertotal) : # termination condition
      X_js = []
      for i in range(len(X_unknown)): # For every unknown node
        X_j = X_unknown[i]
        X_j[0] = X_unknown[i][0] + alpha(N_iter) * levy()
        X_j[1] = X_unknown[i][1] + alpha(N_iter) * levy()
        
        if X_j[0] > X_max or X_j[1] > X_max:
          X_j = [X_max, X_max]

        if X_j[0] > X_max and X_j[1] < X_max:
          X_j = [X_max, X_max]

        if X_j[0] < X_min or X_j[1] < X_min:
          X_j = [X_min, X_min]

        if X_j[0] < X_min and X_j[1] > X_min:
          X_j = [X_min, X_min]



        F_j = objective(X_j[0], X_j[1]) or 0
        rand_k = np.random.randint(0, len(X_unknown))
        F_k = objective(X_unknown[rand_k][0], X_unknown[rand_k][1]) or 0

        if F_j > F_k:
          X_j[0] = X_unknown[rand_k][0]
          X_j[1] = X_unknown[rand_k][1]
          F_j = F_k

        X_js.append(X_j)


      Obj_X_Js = []
      for i in range(len(X_js)):
        Obj_X_Js.append(objective(X_js[i][0], X_js[i][1]))

      # best solution selection
      Obj_X_Js = np.array([np.inf if i is None else i for i in Obj_X_Js])
      F_min = Obj_X_Js[np.argmin(Obj_X_Js)]

      # mutation probability
      Pa_j = []

      for i in Obj_X_Js:
        K = i - F_min
        if K < 1:
          Pa_j.append(pa_min + (pa_max - pa_min) * K)
        else:
          Pa_j.append(pa_max / N_iter)

      # new random solutions generation
      for i in range(len(Pa_j)):
        rand_temp = np.random.uniform(0, 1)
        if rand_temp < Pa_j[i]:
          my_x = np.random.randint(100)
          my_y = np.random.randint(100)
          X_js[i] = [my_x, my_y]



      # finding best solution after replacement with new random solutions
      # Make copy of X_unknown
      X_unknown_copy = X_unknown.copy()
      X_unknown = X_js

        #   Mark
      Obj_X_unknown = []
      for i in range(len(X_unknown)):
        Obj_X_unknown.append(objective(X_unknown[i][0], X_unknown[i][1]))

      Obj_X_unknown = np.array([np.inf if i is None else i for i in Obj_X_unknown])
      F_best_arg = np.argmin(Obj_X_unknown)

      F_best_args = []

      for i in range(len(Obj_X_unknown)):
        if Obj_X_unknown[i] < threshold:
            F_best_args.append(i)

      # Updating Anchor List
      for i in F_best_args:
        X_anchor_temp = list(X_anchor)
        X_anchor_temp.append(X_js[i])
        X_anchor = np.array(X_anchor_temp)

      # Updating Unknown List    
      X_unknown_temp = []
      for i in range(len(X_unknown)):
        if i in F_best_args:
          ales.append([X_unknown[i], X_unknown_copy[i]])
          X_calc.append(X_unknown[i])
        else:
          X_unknown_temp.append(X_unknown[i])


      X_unknown = np.array(X_unknown_temp)


      # 1. When no unknown nodes left are localisable
      if len(X_unknown) == 0:
            break

      N_iter += 1
    # While end
    

    X_unknown = np.array(X_unknown)
    X_calc = np.array(X_calc)
    
    n_iters.append( N_iter )
    n_unknown.append(len(X_unknown))
    
    # Localization Success Ratio
    def lsr():
        return ((len(X_anchor) - M) / len(X_unknown_bak)) * 100
    
    lsr_values.append(lsr())
    
    print(N_iter)
    print(len(X_unknown))
    print(lsr())

0
11
0
100.0
1
8
0
100.0
2
4
0
100.0
3
